<a href="https://colab.research.google.com/github/richmondvan/melanoma-detection/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training

Import all modules and mount Google Drive

In [0]:
# Must be run every time!
import numpy as np
import tensorflow as tf #nightly
import pathlib
from tensorflow.keras import models, layers, losses, metrics
import math
from google.colab import drive 
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

drive.mount('/content/gdrive') 

Prepare datasets

In [0]:
BENIGN_PATH = "/content/gdrive/My Drive/Dataset/DatasetSorted/benign/"
MALIGNANT_PATH = "/content/gdrive/My Drive/Dataset/DatasetSorted/malignant/"

TRAINING_PATH = pathlib.Path("/content/gdrive/My Drive/Dataset/DatasetSorted/training/")
VALIDATION_PATH = pathlib.Path("/content/gdrive/My Drive/Dataset/DatasetSorted/validation/")
TEST_PATH = pathlib.Path("/content/gdrive/My Drive/Dataset/DatasetSorted/test/")

train_image_generator = ImageDataGenerator(
    rescale=1./255,
    brightness_range=(0.95, 1.05),
    horizontal_flip=True,
    vertical_flip=True) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True) # Generator for our validation data

batch_size = 32
epochs = 15
IMG_HEIGHT = 600
IMG_WIDTH = 600

TRAIN_LEN = len(list(TRAINING_PATH.glob("*/*.jpg")))
VALID_LEN = len(list(VALIDATION_PATH.glob("*/*.jpg")))

print(TRAIN_LEN)
print(VALID_LEN)

CLASS_NAMES = ['benign', 'malignant']

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=TRAINING_PATH,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary',
                                                           classes=CLASS_NAMES)

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=VALIDATION_PATH,
                                                              shuffle=True,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary',
                                                              classes=CLASS_NAMES)

Prepare metrics and weights

In [0]:
numBenign = len(list(TRAINING_PATH.glob("benign/*.jpg")))
numMalignant = len(list(TRAINING_PATH.glob("malignant/*.jpg")))
total = numBenign + numMalignant
weight_for_0 = (1 / numBenign)*(total)/2.0 
weight_for_1 = (1 / numMalignant)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)


METRICS = [
      metrics.TruePositives(name='tp'),
      metrics.FalsePositives(name='fp'),
      metrics.TrueNegatives(name='tn'),
      metrics.FalseNegatives(name='fn'), 
      metrics.BinaryAccuracy(name='accuracy'),
      metrics.Precision(name='precision'),
      metrics.Recall(name='recall'),
      metrics.AUC(name='auc'),
]

Prepare model

In [0]:
checkpoint_path = "/content/gdrive/My Drive/Dataset/enet1024_4.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/efficientnet/b7/feature-vector/1", trainable=False),
    layers.Dropout(0.1),
    layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(0.001), activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=METRICS)

model.build([None, 600, 600, 3])
model.summary()


Train model

In [0]:
history = model.fit(x=train_data_gen, epochs=5, initial_epoch=0, verbose=1, validation_data=val_data_gen, validation_steps=VALID_LEN // batch_size, steps_per_epoch=TRAIN_LEN // batch_size, class_weight=class_weight)